In [ ]:
# Copyright 2022 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

使用Vertex可解释人工智能解释图像分类

<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/explainable_ai/xai_image_classification_feature_attributions.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Google Colaboratory logo"><br> 在Colab中打开
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fraw.githubusercontent.com%2FGoogleCloudPlatform%2Fvertex-ai-samples%2Fmain%2Fofficial%2Fexplainable_ai%2fxai_image_classification_feature_attributions.ipynb">
      <img width="32px" src="https://cloud.google.com/ml-engine/images/colab-enterprise-logo-32px.png" alt="Google Cloud Colab Enterprise logo"><br> 在Colab Enterprise中打开
    </a>
  </td>    
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/official/explainable_ai/xai_image_classification_feature_attributions.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo"><br> 在Workbench中打开
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/explainable_ai/xai_image_classification_feature_attributions.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo"><br> 在GitHub上查看
    </a>
  </td>
</table>

## 概述

Vertex Explainable AI提供基于特征和基于示例的解释，以提供更好地理解模型决策过程。对于基于特征的解释，Vertex Explainable AI将特征归因整合到Vertex AI中。特征归因表示模型中每个特征对于每个给定实例的预测贡献了多少。对于图像分类模型，当您请求解释时，您将获得预测的类别以及对于该图像的叠加，显示图像中哪些区域对于最终预测的贡献最大。

要在预训练或自定义训练的模型上使用Vertex Explainable AI，您必须在创建您计划从中请求解释的模型资源时配置某些选项，部署模型时，或提交批量解释作业时。本教程演示了如何配置这些选项，并获取和可视化在线和批量预测的解释。

了解更多关于[Vertex Explainable AI](https://cloud.google.com/vertex-ai/docs/explainable-ai/overview) 和 [Vertex AI Prediction](https://cloud.google.com/vertex-ai/docs/predictions/get-predictions)。

### 目标

在本教程中，您将学习如何在预训练的图像分类模型上配置基于特征的解释，并进行带解释的在线和批量预测。

本教程使用以下 Google Cloud ML 服务和资源：

- Vertex Explainable AI
- Vertex AI Prediction

执行的步骤包括：

- 从 TensorFlow Hub 下载预训练模型
- 上传模型进行部署
- 部署模型进行在线预测
- 进行带解释的在线预测
- 进行带解释的批量预测

### 数据集

在本示例中，您将使用 TensorFlow [flowers](http://download.tensorflow.org/example_images/flower_photos.tgz) 数据集。该数据集包含约3,700张花朵照片，分别保存在五个子目录中，每个目录代表一类花。

成本

本教程使用Google Cloud的可计费组件：

* Vertex AI
* 云存储

了解[Vertex AI价格](https://cloud.google.com/vertex-ai/pricing),
以及[云存储价格](https://cloud.google.com/storage/pricing),
并使用[定价计算器](https://cloud.google.com/products/calculator/)
根据您的预计使用量生成成本估算。

开始吧

### 为Python安装Vertex AI SDK和其他必需的软件包

In [ ]:
# Install the packages
! pip3 install --upgrade -q google-cloud-aiplatform \
                            tensorflow==2.15.1 \
                            tensorflow-hub \
                            matplotlib

重启运行时（仅适用于Colab）

要使用新安装的包，您必须重新启动Google Colab上的运行时。

In [ ]:
import sys

if "google.colab" in sys.modules:

    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

<div class="alert alert-block alert-warning">
<b>⚠️内核将重新启动。在继续下一步之前，请等待直到完成。⚠️</b>
</div>

### 在协作笔记本环境中进行身份验证（仅限Colab）

在谷歌Colab上验证您的环境。

In [ ]:
import sys

if "google.colab" in sys.modules:

    from google.colab import auth

    auth.authenticate_user()

### 设置 Google Cloud 项目信息

了解更多关于[设置项目和开发环境](https://cloud.google.com/vertex-ai/docs/start/cloud-environment)。

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}
LOCATION = "us-central1"  # @param {type:"string"}

### 创建一个云存储桶

创建一个存储桶来存储中间产品，如数据集。

In [ ]:
BUCKET_URI = f"gs://your-bucket-name-{PROJECT_ID}-unique"  # @param {type:"string"}

如果您的桶尚不存在：运行以下单元格以创建您的云存储桶。

In [ ]:
! gsutil mb -l $LOCATION -p $PROJECT_ID $BUCKET_URI

### 导入库

In [ ]:
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
from google.cloud import aiplatform

### 初始化用于 Python 的 Vertex AI SDK

要开始使用 Vertex AI，您必须[启用 Vertex AI API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com)。

In [ ]:
aiplatform.init(project=PROJECT_ID, location=LOCATION, staging_bucket=BUCKET_URI)

## 从TensorFlow Hub下载预训练模型

特征归因支持所有类型的模型（包括AutoML和自定义训练）、框架（TensorFlow、scikit、XGBoost）和模态（图像、文本、表格、视频）。

为了演示目的，本教程使用了来自TensorFlow Hub的图像模型[Inception_v3](https://tfhub.dev/google/imagenet/inception_v3/classification/5)。该模型是在ImageNet基准数据集上进行预训练的。
首先，您可以从TensorFlow Hub下载模型，将其包装为一个Keras层，并将模型工件保存到您的Cloud Storage存储桶中。

In [ ]:
classifier_model = "https://tfhub.dev/google/imagenet/inception_v3/classification/5"

classifier = tf.keras.Sequential([hub.KerasLayer(classifier_model)])

classifier.build([None, 224, 224, 3])

MODEL_DIR = f"{BUCKET_URI}/model"
classifier.save(MODEL_DIR)

上传模型以进行部署

接下来，您需要将模型上传到Vertex AI模型注册表，这将为您的模型创建一个Vertex AI模型资源。在上传之前，您需要定义一个服务函数，将数据转换成模型所期望的格式。

### 为图像数据定义一个serving函数

您可以定义一个serving函数，将图像数据转换为您的模型所期望的格式。当您将编码后的数据发送到Vertex AI时，您的serving函数会确保数据在传递到模型之前在模型服务器上被解码。

要在您的自定义模型中启用Vertex可解释AI，您需要从serving函数中设置两个额外的签名：

- `xai_preprocess`：在serving函数中的预处理函数。
- `xai_model`：用于调用模型的具体函数。

In [ ]:
CONCRETE_INPUT = "numpy_inputs"


def _preprocess(bytes_input):
    decoded = tf.io.decode_jpeg(bytes_input, channels=3)
    decoded = tf.image.convert_image_dtype(decoded, tf.float32)
    resized = tf.image.resize(decoded, size=(224, 224))
    return resized


@tf.function(input_signature=[tf.TensorSpec([None], tf.string)])
def preprocess_fn(bytes_inputs):
    decoded_images = tf.map_fn(
        _preprocess, bytes_inputs, dtype=tf.float32, back_prop=False
    )
    return {
        CONCRETE_INPUT: decoded_images
    }  # User needs to make sure the key matches model's input


@tf.function(input_signature=[tf.TensorSpec([None], tf.string)])
def serving_fn(bytes_inputs):
    images = preprocess_fn(bytes_inputs)
    prob = m_call(**images)
    return prob


m_call = tf.function(classifier.call).get_concrete_function(
    [tf.TensorSpec(shape=[None, 224, 224, 3], dtype=tf.float32, name=CONCRETE_INPUT)]
)

tf.saved_model.save(
    classifier,
    MODEL_DIR,
    signatures={
        "serving_default": serving_fn,
        "xai_preprocess": preprocess_fn,  # Required for XAI
        "xai_model": m_call,  # Required for XAI
    },
)

获取服务函数签名

通过重新加载模型到内存，并查询每个层对应的签名，您可以得到模型输入和输出层的签名。在进行预测请求时，服务函数的输入层名称将会被使用。

In [ ]:
loaded = tf.saved_model.load(MODEL_DIR)

serving_input = list(
    loaded.signatures["serving_default"].structured_input_signature[1].keys()
)[0]
print("Serving function input:", serving_input)

## 配置解释设置

要使用Vertex Explainable AI与自定义训练的模型，您必须在上传模型时配置解释设置。这些设置包括：

- `parameters`：特征归因方法。可用方法包括`shapley`，`ig`，`xrai`。
- `metadata`：用于解释的模型输入和输出。**对于TensorFlow 2模型，此字段是可选的。如果省略，Vertex AI会自动推断模型的输入和输出**。您不需要在本教程中配置此字段。

了解更多关于[配置基于特征的解释](https://cloud.google.com/vertex-ai/docs/explainable-ai/configuring-explanations-feature-based)。

In [ ]:
XAI = "ig"  # [ shapley, ig, xrai ]

if XAI == "shapley":
    PARAMETERS = {"sampled_shapley_attribution": {"path_count": 10}}
elif XAI == "ig":
    PARAMETERS = {"integrated_gradients_attribution": {"step_count": 50}}
elif XAI == "xrai":
    PARAMETERS = {"xrai_attribution": {"step_count": 50}}

parameters = aiplatform.explain.ExplanationParameters(PARAMETERS)

将模型上传到Vertex AI模型资源

接下来，将您的模型上传到一个带有解释配置的Vertex AI模型资源中。 Vertex AI提供[Docker容器映像](https://cloud.google.com/vertex-ai/docs/predictions/pre-built-containers)，您可以将其作为预先构建的容器来运行，以便从训练模型工件中提供预测和解释。

In [ ]:
MODEL_DISPLAY_NAME = "inception-v3-model-unique"
DEPLOY_IMAGE = "us-docker.pkg.dev/vertex-ai/prediction/tf2-cpu.2-11:latest"

model = aiplatform.Model.upload(
    display_name=MODEL_DISPLAY_NAME,
    artifact_uri=MODEL_DIR,
    serving_container_image_uri=DEPLOY_IMAGE,
    explanation_parameters=parameters,
)

将模型部署用于在线预测。

接下来，部署您的模型用于在线预测。您可以设置变量 `DEPLOY_COMPUTE` 来配置用于预测的[计算资源](https://cloud.google.com/vertex-ai/docs/predictions/configure-compute)的机器类型。

In [ ]:
DEPLOY_DISPLAY_NAME = "inception-v3-deploy-unique"
DEPLOY_COMPUTE = "n1-standard-4"

endpoint = model.deploy(
    deployed_model_display_name=DEPLOY_DISPLAY_NAME,
    machine_type=DEPLOY_COMPUTE,
    accelerator_type=None,
    accelerator_count=0,
)

## 用解释进行在线预测

### 下载图像数据集和标签
在这个例子中，您将使用TensorFlow花朵数据集作为预测的输入数据。该数据集包含大约3700张花朵照片，分为五个子目录，每个目录对应一类。您还会获取ImageNet数据集的标签来解码预测结果。

In [ ]:
import pathlib

data_dir = tf.keras.utils.get_file(
    "flower_photos",
    origin="https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz",
    untar=True,
)

data_dir = pathlib.Path(data_dir)
images_files = list(data_dir.glob("daisy/*"))

labels_path = tf.keras.utils.get_file(
    "ImageNetLabels.txt",
    "https://storage.googleapis.com/download.tensorflow.org/data/ImageNetLabels.txt",
)

imagenet_labels = np.array(open(labels_path).read().splitlines())

### 准备图像处理函数

您可以为图像处理和可视化定义一些可重用的函数。

In [ ]:
import base64
import io
import json

import matplotlib.image as mpimg
import matplotlib.pyplot as plt


def encode_image_in_b64(image_file):
    bytes = tf.io.read_file(image_file)
    b64str = base64.b64encode(bytes.numpy()).decode("utf-8")
    return b64str


def decode_b64_image(b64_image_str):
    image = base64.b64decode(b64_image_str)
    image = io.BytesIO(image)
    image = mpimg.imread(image, format="JPG")
    return image


def decode_numpy_image(numpy_inputs):
    numpy_inputs_json = json.loads(str(numpy_inputs))
    image = np.array(numpy_inputs_json)
    return image


def show_explanation(encoded_image, prediction, feature_attributions):
    fig, axs = plt.subplots(nrows=1, ncols=3, figsize=(16, 4))

    label_index = np.argmax(prediction)
    class_name = imagenet_labels[label_index]
    confidence_score = prediction[label_index]
    axs[0].set_title(
        "Prediction:[" + class_name + "] (" + str(round(confidence_score, 1)) + "%)"
    )
    original_image = decode_b64_image(encoded_image)
    axs[0].imshow(original_image, interpolation="nearest", aspect="auto")
    axs[0].axis("off")

    numpy_inputs = feature_attributions["numpy_inputs"]
    attribution_image = decode_numpy_image(numpy_inputs)
    axs[1].set_title("Feature attributions")
    axs[1].imshow(attribution_image, interpolation="nearest", aspect="auto")
    axs[1].axis("off")

    processed_image = attribution_image.max(axis=2)
    axs[2].imshow(processed_image, cmap="coolwarm", aspect="auto")
    axs[2].set_title("Feature attribution heatmap")
    axs[2].axis("off")

获取在线解释

您向`endpoint`发送一个带有编码输入图像数据的`explain`请求，并获取带有解释的预测结果。

In [ ]:
TEST_IMAGE_SIZE = 2

test_image_list = []
for i in range(TEST_IMAGE_SIZE):
    test_image_list.append(str(images_files[i]))

instances_list = []
for test_image in test_image_list:
    b64str = encode_image_in_b64(test_image)
    instances_list.append({serving_input: {"b64": b64str}})

response = endpoint.explain(instances_list)

### 可视化在线解释

当您请求对图像分类模型的解释时，您将获得预测类别以及一个图像叠加显示哪些像素（集成梯度）或区域（集成梯度或XRAI）有助于预测的解释。

In [ ]:
for i, test_image in enumerate(test_image_list):
    encoded_image = encode_image_in_b64(test_image)
    prediction = response.predictions[i]
    explanation = response.explanations[i]
    feature_attributions = dict(explanation.attributions[0].feature_attributions)

    show_explanation(encoded_image, prediction, feature_attributions)

## 使用解释进行批量预测

### 创建批量输入文件

您可以创建一个以JSONL格式的批量输入文件，并将输入文件存储在您的云存储桶中。

In [ ]:
gcs_input_uri = f"{BUCKET_URI}/test_images.json"

with tf.io.gfile.GFile(gcs_input_uri, "w") as f:
    for test_image in test_image_list:
        b64str = encode_image_in_b64(test_image)
        data = {serving_input: {"b64": b64str}}
        f.write(json.dumps(data) + "\n")

提交一个批量预测作业

通过将`generate_explanation`参数设置为`True`来提交一个批量预测作业。

In [ ]:
JOB_DISPLAY_NAME = "inception-v3-job-unique"

batch_predict_job = model.batch_predict(
    job_display_name=JOB_DISPLAY_NAME,
    gcs_source=gcs_input_uri,
    gcs_destination_prefix=BUCKET_URI,
    instances_format="jsonl",
    model_parameters=None,
    machine_type=DEPLOY_COMPUTE,
    generate_explanation=True,
)

### 获取批量解释

接下来，您将从完成的批量预测作业中获取解释。结果将被写入您在批量预测请求中指定的Cloud Storage输出桶中。您可以调用方法`iter_outputs()`来获取生成的结果中每个Cloud Storage文件的列表。

In [ ]:
bp_iter_outputs = batch_predict_job.iter_outputs()

explanation_results = list()
for blob in bp_iter_outputs:
    if blob.name.split("/")[-1].startswith("explanation.results"):
        explanation_results.append(blob.name)

### 可视化解释

您可以将一个解释结果作为例子，然后可视化解释。对于一种图像分类模型，您可以获得预测类别以及一个图像叠加层，显示哪些像素（整合梯度）或区域（整合梯度或XRAI）对预测做出了贡献。

In [ ]:
explanation_result = explanation_results[0]

gfile_name = f"{BUCKET_URI}/{explanation_result}"
with tf.io.gfile.GFile(name=gfile_name, mode="r") as gfile:
    result = json.loads(gfile.read())

encoded_image = result["instance"]["bytes_inputs"]["b64"]
prediction = result["prediction"]
attributions = result["explanation"]["attributions"][0]
feature_attributions = attributions["featureAttributions"]

show_explanation(encoded_image, prediction, feature_attributions)

清理

要清理此项目中使用的所有Google Cloud资源，您可以[删除用于本教程的Google Cloud项目](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects)。

否则，您可以删除本教程中创建的各个资源。

In [ ]:
# Deploy the model from endpoint
endpoint.undeploy_all()
# Delete the endpoint
endpoint.delete()
# Delete the model
model.delete()
# Delete the batch prediction job
batch_predict_job.delete()

# Delete Cloud Storage bucket
delete_bucket = False  # Set True to delete your bucket
if delete_bucket:
    ! gsutil -m rm -r $BUCKET_URI